# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Input File (CSV)
input_data_file = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read our Data file with the pandas library
weather_data_df = pd.read_csv(input_data_file, encoding="ISO-8859-1")
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mattru,7.6244,-11.8332,73.42,85,54,2.84,SL,1612581797
1,Hobart,-42.8794,147.3294,78.80,41,40,18.41,AU,1612581712
2,Hermanus,-34.4187,19.2345,59.00,92,4,4.63,ZA,1612581536
3,Atuona,-9.8000,-139.0333,79.57,81,74,18.79,PF,1612581564
4,KlaksvÃ­k,62.2266,-6.5890,35.60,69,75,12.66,FO,1612581221


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data_df[["Lat", "Lng"]]
print(locations)

# Convert to float
humidity_weights = weather_data_df["Humidity"].astype(float)
humidity_weights

         Lat       Lng
0     7.6244  -11.8332
1   -42.8794  147.3294
2   -34.4187   19.2345
3    -9.8000 -139.0333
4    62.2266   -6.5890
..       ...       ...
604  -2.0833  101.3833
605 -34.5703  -59.1050
606  54.8167   54.0000
607  63.0500  179.3167
608 -15.1961   12.1522

[609 rows x 2 columns]


0      85.0
1      41.0
2      92.0
3      81.0
4      69.0
       ... 
604    53.0
605    96.0
606    97.0
607    87.0
608    83.0
Name: Humidity, Length: 609, dtype: float64

In [8]:
# Plot Heatmap
fig = gmaps.figure(center=(20, 20), zoom_level=1.8)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weights, 
                                 dissipating=False, max_intensity=30,
                                 point_radius=1.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
cities_df = weather_data_df.copy()
cities_df.drop(cities_df[(cities_df['Max Temp'] < 75)].index, inplace = True) 
cities_df.drop(cities_df[(cities_df['Humidity'] > 60)].index, inplace = True) 
cities_df.drop(cities_df[(cities_df['Cloudiness'] > 40)].index, inplace = True) 
cities_df.drop(cities_df[(cities_df['Wind Speed'] > 5)].index, inplace = True) 
cities_df.dropna(inplace = True)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
93,Chuncheng,22.1731,111.7886,77.00,46,0,2.73,CN,1612581918
156,Magway,20.1500,94.9167,79.16,35,0,1.10,MM,1612582000
243,Sikasso,10.9167,-7.0000,75.33,21,34,0.85,ML,1612582114
250,San Juan,-31.5375,-68.5364,79.70,35,0,4.14,AR,1612582123
261,Panguna,-6.3164,155.4848,84.60,57,18,3.44,PG,1612582137
355,Svay Rieng,11.0879,105.7993,86.61,51,39,4.41,KH,1612582262
382,Sibolga,1.7427,98.7792,86.58,56,0,1.50,ID,1612582182
450,Dawei,14.0833,98.2000,89.26,33,36,4.32,MM,1612582383


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = cities_df.copy().reset_index(drop=True)
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chuncheng,22.1731,111.7886,77.00,46,0,2.73,CN,1612581918,
1,Magway,20.1500,94.9167,79.16,35,0,1.10,MM,1612582000,
2,Sikasso,10.9167,-7.0000,75.33,21,34,0.85,ML,1612582114,
3,San Juan,-31.5375,-68.5364,79.70,35,0,4.14,AR,1612582123,
4,Panguna,-6.3164,155.4848,84.60,57,18,3.44,PG,1612582137,
5,Svay Rieng,11.0879,105.7993,86.61,51,39,4.41,KH,1612582262,
6,Sibolga,1.7427,98.7792,86.58,56,0,1.50,ID,1612582182,
7,Dawei,14.0833,98.2000,89.26,33,36,4.32,MM,1612582383,


In [11]:
# Set parameters to search for hotels with 5000 meters.
target_type = "lodging"

# city_coords = "35.1963662,-80.8941772"
radius = 5000

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# for index, row in temp_df.iterrows():
for index, row in hotel_df.iterrows():
    city_coords = f"{row['Lat']},{row['Lng']}"

    # Write params dict
    params = {
        "location": city_coords,
        "types": target_type,
        "radius": radius,
        "key": g_key
    }


    # Run request
    response = requests.get(base_url, params)

    hotels_json = response.json()
    if hotels_json['status'] != "ZERO_RESULTS":
        try:
            hotel_df.iloc[index,len(hotel_df.columns)-1] = hotels_json["results"][0]["name"]
        except IndexError as e: 
            print(e)
            print(json.dumps(hotels_json, indent=4))
    else:
        hotel_df.iloc[index,len(hotel_df.columns)-1] = "N/A"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chuncheng,22.1731,111.7886,77.00,46,0,2.73,CN,1612581918,7Days Inn Yangjiang Yangchun Bus Terminal
1,Magway,20.1500,94.9167,79.16,35,0,1.10,MM,1612582000,Htein Htein Thar 2
2,Sikasso,10.9167,-7.0000,75.33,21,34,0.85,ML,1612582114,N/A
3,San Juan,-31.5375,-68.5364,79.70,35,0,4.14,AR,1612582123,Hotel Provincial
4,Panguna,-6.3164,155.4848,84.60,57,18,3.44,PG,1612582137,N/A
5,Svay Rieng,11.0879,105.7993,86.61,51,39,4.41,KH,1612582262,Apsara guesthouse
6,Sibolga,1.7427,98.7792,86.58,56,0,1.50,ID,1612582182,Hotel Syariah CN Darussalam
7,Dawei,14.0833,98.2000,89.26,33,36,4.32,MM,1612582383,Hotel Dawei


In [12]:
hotel_df.drop(hotel_df[(hotel_df['Hotel Name'] == "N/A")].index, inplace = True)
hotel_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chuncheng,22.1731,111.7886,77.00,46,0,2.73,CN,1612581918,7Days Inn Yangjiang Yangchun Bus Terminal
1,Magway,20.1500,94.9167,79.16,35,0,1.10,MM,1612582000,Htein Htein Thar 2
2,San Juan,-31.5375,-68.5364,79.70,35,0,4.14,AR,1612582123,Hotel Provincial
3,Svay Rieng,11.0879,105.7993,86.61,51,39,4.41,KH,1612582262,Apsara guesthouse
4,Sibolga,1.7427,98.7792,86.58,56,0,1.50,ID,1612582182,Hotel Syariah CN Darussalam
5,Dawei,14.0833,98.2000,89.26,33,36,4.32,MM,1612582383,Hotel Dawei


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))